## 단어의 등장 순서를 고려하지 않는 빈도수 기반 단어 표현 방법인 Bag of Words에 대해서 배운다.

### 1. Bag of Words란

단어들의 순서는 고려하지 않고, 단어들의 frequency에만 집중한다.    

1. 각 단어에 고유한 정수 인덱스를 부여한다.  
2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다.

문서 1 : 정부가 발표하는 물가상승률과 소비자가 느기는 물가상승률은 다르다.  

에 대해서 BoW를 만들어보자.

In [1]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
    # 온점 제거 및 형태소 분석
    document = document.replace('.')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            # Bow에 전부 기본값 1를 넣는다.
            bow.insert(len(word_to_index)-1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다
            bow[index] = bow[index] + 1

    return word_to_index, bow


JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.